# Unpack raw Data

In this notebook we will unpack the raw data from the Eurostat database. The raw data is stored in the `data/raw` folder. The unpacked dataframes will be stored in the `data/dataframes` folder.

The nested index of the tables is ungrouped to reveal the categorical information. The dataframes are stored as csv files. 

In [1]:
import pandas as pd
import numpy as np
import regex as re

LOAD DATA

In [2]:
datasets = ['area', 'owner', 'irspec', 'vol', 'remov']
category = 'forest'

In [3]:
# Loading data

def create_df (dataset):

    data = pd.read_csv (f"../data/raw/{category}/for_{dataset}_tabular.tsv", sep = '\t')
    print('\n', dataset)
    print('  Size:    ', data.shape)
    print('  Idex:    ', data.columns[0])
    print('  Columns: ', data.columns[1:][::-3])
    data.name = str(dataset)
    return data

frames = []
for dataset in datasets:
    frames.append(create_df(dataset))


 area
  Size:     (62, 10)
  Idex:     freq,unit,indic_fo,geo\TIME_PERIOD
  Columns:  Index(['2020 ', '2017 ', '2010 '], dtype='object')

 owner
  Size:     (279, 31)
  Idex:     freq,unit,owner,treespec,geo\TIME_PERIOD
  Columns:  Index(['2021 ', '2018 ', '2015 ', '2012 ', '2009 ', '2006 ', '2003 ', '2000 ',
       '1997 ', '1994 '],
      dtype='object')

 irspec
  Size:     (2066, 31)
  Idex:     freq,treespec,prod_wd,stk_flow,unit,geo\TIME_PERIOD
  Columns:  Index(['2021 ', '2018 ', '2015 ', '2012 ', '2009 ', '2006 ', '2003 ', '2000 ',
       '1997 ', '1994 '],
      dtype='object')

 vol
  Size:     (47, 10)
  Idex:     freq,unit,indic_fo,geo\TIME_PERIOD
  Columns:  Index(['2020 ', '2017 ', '2010 '], dtype='object')

 remov
  Size:     (1170, 35)
  Idex:     freq,treespec,prod_wd,unit,bark,geo\TIME_PERIOD
  Columns:  Index(['2021 ', '2018 ', '2015 ', '2012 ', '2009 ', '2006 ', '2003 ', '2000 ',
       '1997 ', '1994 ', '1991 ', '1988 '],
      dtype='object')


UNGROUP INDEX COLUMN

In [4]:
def index_split(frame):
    
    print(frame.name, '    original shape:', frame.shape)
    frame = frame.copy()

    categories = frame.columns[0].split(',')
    print('  categories: ', '\n        ', categories)
    frame.rename(columns={ frame.columns[0]: "idx" }, inplace = True)
    n_cats = len(categories)
    
    frame.idx = frame.idx.str.split(',').tolist()
  
    for i, category in enumerate(categories):
        frame[category] = [str(x[::--1][i]) for x in frame.idx]
    
    frame.idx = [','.join(x) for x in frame.idx]
    
    # swap columns
    frame = frame[['idx'] + categories + frame.columns[1:-n_cats][::-3].tolist()]
    print('  new columns: ', '\n        ', frame.columns.values, '\n')
        
    return frame

ungrouped = [index_split(frame) for frame in frames]

print('\n Example:', datasets[0], ungrouped[0].shape)
ungrouped[0].head()

area     original shape: (62, 10)
  categories:  
         ['freq', 'unit', 'indic_fo', 'geo\\TIME_PERIOD']
  new columns:  
         ['idx' 'freq' 'unit' 'indic_fo' 'geo\\TIME_PERIOD' '2020 ' '2017 ' '2010 '] 

owner     original shape: (279, 31)
  categories:  
         ['freq', 'unit', 'owner', 'treespec', 'geo\\TIME_PERIOD']
  new columns:  
         ['idx' 'freq' 'unit' 'owner' 'treespec' 'geo\\TIME_PERIOD' '2021 ' '2018 '
 '2015 ' '2012 ' '2009 ' '2006 ' '2003 ' '2000 ' '1997 ' '1994 '] 

irspec     original shape: (2066, 31)
  categories:  
         ['freq', 'treespec', 'prod_wd', 'stk_flow', 'unit', 'geo\\TIME_PERIOD']
  new columns:  
         ['idx' 'freq' 'treespec' 'prod_wd' 'stk_flow' 'unit' 'geo\\TIME_PERIOD'
 '2021 ' '2018 ' '2015 ' '2012 ' '2009 ' '2006 ' '2003 ' '2000 ' '1997 '
 '1994 '] 

vol     original shape: (47, 10)
  categories:  
         ['freq', 'unit', 'indic_fo', 'geo\\TIME_PERIOD']
  new columns:  
         ['idx' 'freq' 'unit' 'indic_fo' 'geo\\TIME_PERIOD

,idx,freq,unit,indic_fo,geo\TIME_PERIOD,2020,2017,2010
0,"A,THS_HA,FOR,AT",A,THS_HA,FOR,AT,3899.15,3888.38,3863.20
1,"A,THS_HA,FOR,BA",A,THS_HA,FOR,BA,2187.91,2187.91,2102.66
2,"A,THS_HA,FOR,BE",A,THS_HA,FOR,BE,689.30,689.30,689.87
3,"A,THS_HA,FOR,BG",A,THS_HA,FOR,BG,3893.00,3854.00,3737.00
4,"A,THS_HA,FOR,CH",A,THS_HA,FOR,CH,1269.11,1258.79,1234.72


Clean the Data
- delete columns without relevant information
- rename columns

In [5]:
def remove_affixes(x):
    reduced =           str(x).translate({ord(i): '' for i in ' edp:cs'})
    if reduced == '':   return np.nan
    else:               return reduced


def drop_cols(frame, name):
    print(name, '   original shape:', frame.shape)
    frame = frame.copy()
    
    # rename columns
    frame.rename(columns={'geo\TIME_PERIOD': "Country"}, inplace = True)
    print('         ', *frame.columns.values) 
    print('         ', *frame.dtypes)
       
    # clean nan-formatting
    frame.replace(': ', np.nan, inplace=True)

    # repair float columns for year-columns (containing the values)
    for col in frame.columns.values.tolist():
        if bool(re.match(r'\d{4}', col)) & (frame[col].dtype == 'object'):
            frame[col] = [remove_affixes(x) for x in frame[col]]
            frame[col] = frame[col].astype(float)
            
    # drop columns with only one unique value (no info value)
    for col in frame.columns:        
        if frame[f'{col}'].nunique() == 1:
            frame.drop(col, axis=1, inplace=True)
            
    print('\n        new shape: ', frame.shape)   
    print('         ', *frame.columns.values)    
    print('         ', *frame.dtypes, '\n') 
       
    return frame

clean_frames = [drop_cols(frame, name) for frame, name in zip(ungrouped, datasets)]

area    original shape: (62, 8)
          idx freq unit indic_fo Country 2020  2017  2010 
          object object object object object float64 float64 float64

        new shape:  (62, 6)
          idx indic_fo Country 2020  2017  2010 
          object object object float64 float64 float64 

owner    original shape: (279, 16)
          idx freq unit owner treespec Country 2021  2018  2015  2012  2009  2006  2003  2000  1997  1994 
          object object object object object object object object object object object object object object object object

        new shape:  (279, 14)
          idx owner treespec Country 2021  2018  2015  2012  2009  2006  2003  2000  1997  1994 
          object object object object float64 float64 float64 float64 float64 float64 float64 float64 float64 float64 

irspec    original shape: (2066, 17)
          idx freq treespec prod_wd stk_flow unit Country 2021  2018  2015  2012  2009  2006  2003  2000  1997  1994 
          object object object object 

In [6]:
n = 2
print('\n Example:', datasets[n], clean_frames[n].shape)
clean_frames[n]


 Example: irspec (2066, 15)


,idx,treespec,stk_flow,unit,Country,2021,2018,2015,2012,2009,2006,2003,2000,1997,1994
0,"A,CONIF,RW_IN,EXP,THS_EUR,AT",CONIF,EXP,THS_EUR,AT,77676.78,64253.07,57954.00,57941.00,46230.00,41286.00,39882.00,34439.00,37873.18,33743.52
1,"A,CONIF,RW_IN,EXP,THS_EUR,BE",CONIF,EXP,THS_EUR,BE,178707.58,65139.62,48734.92,53814.79,30001.33,36643.00,37785.02,46916.00,NaN,NaN
2,"A,CONIF,RW_IN,EXP,THS_EUR,BG",CONIF,EXP,THS_EUR,BG,6214.88,15695.62,4358.73,7528.87,4357.04,6218.94,2600.77,3147.12,304.73,266.30
3,"A,CONIF,RW_IN,EXP,THS_EUR,CH",CONIF,EXP,THS_EUR,CH,25252.06,42683.98,39797.73,55504.85,46423.84,91625.98,76189.98,141398.68,56586.97,53996.47
4,"A,CONIF,RW_IN,EXP,THS_EUR,CY",CONIF,EXP,THS_EUR,CY,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,145.36,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2061,"A,TOTAL,RW_IN,IMP,THS_NAC,RO",TOTAL,IMP,THS_NAC,RO,979865.47,419244.35,457925.11,192538.42,86315.74,90276.54,4790.53,287059.38,941.87,363.13
2062,"A,TOTAL,RW_IN,IMP,THS_NAC,SE",TOTAL,IMP,THS_NAC,SE,3648017.00,6391930.00,3443491.00,3166144.00,2282152.00,2979436.00,3613479.00,4041340.00,2733672.37,2438555.16
2063,"A,TOTAL,RW_IN,IMP,THS_NAC,SI",TOTAL,IMP,THS_NAC,SI,72972.23,33918.48,31414.36,21609.99,15473.49,30914.73,29537.12,28033.20,8696.16,8852.31
2064,"A,TOTAL,RW_IN,IMP,THS_NAC,SK",TOTAL,IMP,THS_NAC,SK,172033.18,56013.82,19436.43,63735.04,19685.98,22484.20,6047.77,27838.77,5678.10,5373.58


In [7]:
for i in range(len(datasets)):
    print('\n', datasets[i], clean_frames[i].shape)
    print(clean_frames[i].dtypes, '\n')
    print(clean_frames[i].describe(), '\n')


 area (62, 6)
idx          object
indic_fo     object
Country      object
2020        float64
2017        float64
2010        float64
dtype: object 

              2020          2017          2010 
count     62.000000     62.000000     62.000000
mean    3251.208548   3239.123226   3199.315645
std     5741.959598   5728.478555   5686.036829
min        0.000000      0.000000      0.000000
25%       43.237500     47.422500     47.095000
50%      812.510000    790.910000    764.000000
75%     3061.772500   3040.915000   2958.595000
max    27980.000000  27980.000000  28073.000000 


 owner (279, 14)
idx          object
owner        object
treespec     object
Country      object
2021        float64
2018        float64
2015        float64
2012        float64
2009        float64
2006        float64
2003        float64
2000        float64
1997        float64
1994        float64
dtype: object 

              2021          2018          2015          2012          2009   \
count    168.000000  1

SAVE CSV DATAFRAMES

In [8]:
def dataframing(frames, datasets):
    for i in range(len(datasets)):
        frames[i].to_csv(f'../data/dataframes/{category}/for_{datasets[i]}.csv', index = False)
    
dataframing(clean_frames, datasets)   